In [1]:
import torch

inputs = torch.tensor(
    [
        [0.72, 0.45, 0.31],# 'Dream' (x^1)
        [0.75, 0.20, 0.55],# 'big'   (x^2)
        [0.30, 0.80, 0.40],# 'and'   (x^3)
        [0.85, 0.35, 0.60],# 'work'  (x^4)
        [0.55, 0.15, 0.75],# 'for'   (x^5)
        [0.25, 0.20, 0.85],# 'it'    (x^6)
    ]
)

# Corresponding words
words = ['Dream', 'big', 'and', 'work', 'for', 'it']

In [ ]:
import torch.nn as nn

class CausalAttention(nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout, qkv_bias=False):
        super().__init__()
        self.d_out = d_out
        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.dropout = nn.Dropout(dropout)
        self.register_buffer('mask', torch.triu(torch.ones(context_length, context_length), diagonal=1))
    
    def forward(self, x):
        b, num_tokens, d_in = x.shape
        queries = self.W_query(x)
        keys = self.W_key(x)
        values = self.W_value(x)
        attention_scores = queries @ keys.transpose(1, 2)
        attention_scores.masked_fill_(self.mask.bool()[:num_tokens, :num_tokens], -torch.inf)
        attention_weights = torch.softmax(attention_scores / keys.shape[-1]**0.5, dim=-1)
        attention_weights = self.dropout(attention_weights)
        context_vector = attention_weights @ values
        return context_vector

In [3]:
class MultiHeadAttentionWrapper(nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias=False):
        super().__init__()
        self.heads = nn.ModuleList(
            [CausalAttention(d_in, d_out, context_length, dropout, qkv_bias) for _ in range(num_heads)]
        )
    
    def forward(self, x):
        return torch.cat([head(x) for head in self.heads], dim=-1)

In [4]:
d_in = inputs.shape[-1]
d_out = 2
print(d_in, d_out)

3 2


In [5]:
batch = torch.stack((inputs,inputs), dim=0)
print(batch.shape)

torch.Size([2, 6, 3])


In [6]:
torch.manual_seed(123)
context_length = batch.shape[1]
mha = MultiHeadAttentionWrapper(d_in, d_out, context_length, 0.0, num_heads=2)


In [7]:
context_vectors = mha(batch)
print('Context Vectors')
print(context_vectors)
print(context_vectors.shape)

Context Vectors
tensor([[[-0.5762, -0.1627,  0.5569,  0.3635],
         [-0.5650, -0.0630,  0.5599,  0.3006],
         [-0.5472, -0.1226,  0.5285,  0.3435],
         [-0.5787, -0.0943,  0.5621,  0.3388],
         [-0.5593, -0.0436,  0.5509,  0.3046],
         [-0.5287, -0.0033,  0.5277,  0.2743]],

        [[-0.5762, -0.1627,  0.5569,  0.3635],
         [-0.5650, -0.0630,  0.5599,  0.3006],
         [-0.5472, -0.1226,  0.5285,  0.3435],
         [-0.5787, -0.0943,  0.5621,  0.3388],
         [-0.5593, -0.0436,  0.5509,  0.3046],
         [-0.5287, -0.0033,  0.5277,  0.2743]]], grad_fn=<CatBackward0>)
torch.Size([2, 6, 4])
